In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from glob import glob
from tqdm import tqdm
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from shutil import copy
from sklearn.model_selection import StratifiedKFold
import torch
from tempfile import TemporaryDirectory

from albk.data.utils import idx_to_locate
use_disjoint_files = False


import torch
import torch.nn as nn

from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed
from itertools import product
import torch
import torch.nn as nn
from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import product
from torch.utils.data import TensorDataset, DataLoader
from astra.torch.models import EfficientNetClassifier,EfficientNet_B0_Weights
from astra.torch.utils import train_fn

import torchvision.models as models
from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score,classification_report

In [2]:
# path='/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/new_dataset_delhi_9k.pt' 
# loaded_data = torch.load(path)
# images = loaded_data['images']
# labels = loaded_data['labels'] 
# images.shape, labels.shape

In [3]:
# images.dtype, labels.dtype
# labels = labels.type(torch.uint8)
# images.dtype, labels.dtype


In [4]:
# import torch
from collections import Counter

fold_data = []  # List to store data from each fold

seed = 42  # Use your desired random seed
# splitter = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
# # images = images / 255
# #     # mean normalize
# # images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)
# for fold, (train_idx, test_idx) in enumerate(splitter.split(images, labels)):
#     X_train, X_test = images[train_idx], images[test_idx]
#     y_train, y_test = labels[train_idx], labels[test_idx]

#     # Count occurrences of each class in train and test sets
#     train_counter = Counter(y_train.numpy())
#     test_counter = Counter(y_test.numpy())
#     print(train_counter)
#     print(test_counter)
#     print(f"Fold {fold + 1} - Train: {train_counter}, Test: {test_counter}")

#     fold_data.append({
#         'fold': fold + 1,
#         'X_train': X_train,
#         'X_test': X_test,
#         'y_train': y_train,
#         'y_test': y_test,
#         'train_counter': train_counter,
#         'test_counter': test_counter
#     })


In [5]:
# test_images = fold_data[0]['X_test']
# test_labels = fold_data[0]['y_test']
# print(test_images.shape, test_labels.shape)

In [6]:
# save_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/test_data.pt"
# torch.save({
#     'images': test_images,
#     'labels': test_labels
# }, save_path)

In [7]:
# train_images = fold_data[0]['X_train']
# train_labels = fold_data[0]['y_train']
# print(train_images.shape, train_labels.shape)

In [8]:
# save_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/delhi_data_4500.pt"
# torch.save({
#     'images': train_images,
#     'labels': train_labels
# }, save_path)

In [9]:
path='/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/delhi_data_4500.pt'
loaded_data = torch.load(path)
images45 = loaded_data['images']
labels45 = loaded_data['labels']
images45.shape, labels45.shape

(torch.Size([4500, 3, 224, 224]), torch.Size([4500]))

In [64]:
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
# images = images / 255
#     # mean normalize
# images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)
for fold, (train_idx, test_idx) in enumerate(splitter.split(images45, labels45)):
    X_train, X_test = images45[train_idx], images45[test_idx]
    y_train, y_test = labels45[train_idx], labels45[test_idx]

    # Count occurrences of each class in train and test sets
    train_counter = Counter(y_train.numpy())
    test_counter = Counter(y_test.numpy())
    print(train_counter)
    print(test_counter)
    print(f"Fold {fold + 1} - Train: {train_counter}, Test: {test_counter}")

    fold_data.append({
        'fold': fold + 1,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'train_counter': train_counter,
        'test_counter': test_counter
    })

Counter({0: 3184, 1: 416})
Counter({0: 795, 1: 105})
Fold 1 - Train: Counter({0: 3184, 1: 416}), Test: Counter({0: 795, 1: 105})
Counter({0: 3183, 1: 417})
Counter({0: 796, 1: 104})
Fold 2 - Train: Counter({0: 3183, 1: 417}), Test: Counter({0: 796, 1: 104})
Counter({0: 3183, 1: 417})
Counter({0: 796, 1: 104})
Fold 3 - Train: Counter({0: 3183, 1: 417}), Test: Counter({0: 796, 1: 104})
Counter({0: 3183, 1: 417})
Counter({0: 796, 1: 104})
Fold 4 - Train: Counter({0: 3183, 1: 417}), Test: Counter({0: 796, 1: 104})
Counter({0: 3183, 1: 417})
Counter({0: 796, 1: 104})
Fold 5 - Train: Counter({0: 3183, 1: 417}), Test: Counter({0: 796, 1: 104})


In [11]:
from sklearn.model_selection import train_test_split
train_25_image,_,train_25_y,_ = train_test_split(images45,labels45, train_size=0.10, random_state=42, stratify=labels45)
train_25_image.shape, train_25_y.shape

(torch.Size([450, 3, 224, 224]), torch.Size([450]))

In [12]:
save_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/delhi_data_450.pt"
torch.save({
    'images': train_25_image,
    'labels': train_25_y
}, save_path)

In [34]:
# save_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/delhi_data_2250.pt"
# torch.save({
#     'images': test_images,
#     'labels': test_labels
# }, save_path)

In [43]:
train_images = fold_data[0]['X_train']
train_labels = fold_data[0]['y_train']
print(train_images.shape, train_labels.shape)

torch.Size([2250, 3, 224, 224]) torch.Size([2250])


In [37]:
# images45 = train_images
# labels45 = train_labels
# images45.shape, labels45.shape

(torch.Size([2250, 3, 224, 224]), torch.Size([2250]))

In [44]:
splitter = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
# images = images / 255
#     # mean normalize
# images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)
for fold, (train_idx, test_idx) in enumerate(splitter.split(train_images, train_labels)):
    X_train, X_test = train_images[train_idx], train_images[test_idx]
    y_train, y_test = train_labels[train_idx], train_labels[test_idx]

    # Count occurrences of each class in train and test sets
    train_counter = Counter(y_train.numpy())
    test_counter = Counter(y_test.numpy())
    print(train_counter)
    print(test_counter)
    print(f"Fold {fold + 1} - Train: {train_counter}, Test: {test_counter}")

    fold_data.append({
        'fold': fold + 1,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'train_counter': train_counter,
        'test_counter': test_counter
    })

Counter({0: 995, 1: 130})
Counter({0: 995, 1: 130})
Fold 1 - Train: Counter({0: 995, 1: 130}), Test: Counter({0: 995, 1: 130})
Counter({0: 995, 1: 130})
Counter({0: 995, 1: 130})
Fold 2 - Train: Counter({0: 995, 1: 130}), Test: Counter({0: 995, 1: 130})


In [49]:
images = fold_data[0]['X_train']
labels = fold_data[0]['y_train']
print(images.shape, labels.shape)

torch.Size([2250, 3, 224, 224]) torch.Size([2250])


In [48]:
images.shape, labels.shape

(torch.Size([2250, 3, 224, 224]), torch.Size([2250]))

In [ ]:
save_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/delhi_data_.pt"
torch.save({
    'images': test_images,
    'labels': test_labels
}, save_path)